<a href="https://colab.research.google.com/github/enoosoft/ai_fsec/blob/master/colab/1_mnist_softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Lab 10 MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms # 이미지를 행렬 값으로 변환
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
# parameters
learning_rate = 0.001 # 학습률
training_epochs = 15 # 전체데이터 학습을 15번 반복 하겠다
batch_size = 100 # 5만 * 15 = 75만이 동시에 들어가면 OOM 이 나고 속도도 저하된다. 부하를 나누고 성능을 향상하기 위해 fetch size 를 지정한다 

In [4]:
# MNIST dataset. training set, test set 을 나누는 이유?
# 데이터를 모두 가지고 있지 않은 경우, 학습집합(training)이 아닌 미학습집합(test)에 대한 정확도를 평가하는 것이 더 중요하다.
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [5]:
# dataset loader 학습집합
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True, # 뒤섞어 뽑는다. 실제 상황은 랜덤에 가깝기 때문에.
                                          drop_last=True) # 자투리를 버리겠다.

In [6]:
# MNIST data image of shape 28 * 28 = 784 mnist 1개
linear = torch.nn.Linear(784, 10, bias=True).to(device) # 픽셀수 784 -> 출력층 아라비아 숫자 10개 (0,1,2,3,4,5,6,7,8,9)

In [7]:
# Initialization
torch.nn.init.normal_(linear.weight) # 정규분포로 초기화 하겠다. edge(선)이 7840개가 되는데 이것을 정규분포로 설정

Parameter containing:
tensor([[-1.2901,  0.3930, -0.1891,  ..., -0.1626, -0.3275,  0.5608],
        [-1.6757, -0.2454,  0.9221,  ...,  0.9559,  0.6160, -0.4226],
        [ 0.2444,  1.4810, -2.0906,  ...,  0.1206,  1.0044, -0.3737],
        ...,
        [ 0.6234,  1.8019, -2.7563,  ..., -0.5889, -0.5576,  0.7360],
        [-0.2871, -1.3313, -2.2248,  ...,  0.0309,  0.9180, -0.1482],
        [ 0.7678,  0.6624, -0.5362,  ...,  0.2338,  0.3688, -0.7182]],
       device='cuda:0', requires_grad=True)

In [8]:
# define cost/loss & optimizer. 소프트맥스는 CrossEntropyLoss(다중 분류)
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(linear.parameters(), lr=learning_rate) # SGD(경사하강법)아닌 Adam을 상용(SDG보다 더좋다는)

In [9]:
total_batch = len(data_loader)
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # reshape input image into [batch_size by 784]
        # label is not one-hot encoded
        X = X.view(-1, 28 * 28).to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = linear(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning finished')

Epoch: 0001 cost = 5.645695210
Epoch: 0002 cost = 1.696827292
Epoch: 0003 cost = 1.121626616
Epoch: 0004 cost = 0.882780135
Epoch: 0005 cost = 0.748860061
Epoch: 0006 cost = 0.661943853
Epoch: 0007 cost = 0.599809587
Epoch: 0008 cost = 0.553705037
Epoch: 0009 cost = 0.517529070
Epoch: 0010 cost = 0.488772631
Epoch: 0011 cost = 0.465197325
Epoch: 0012 cost = 0.445370257
Epoch: 0013 cost = 0.428630620
Epoch: 0014 cost = 0.413612187
Epoch: 0015 cost = 0.401216000
Learning finished


In [10]:
# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = linear(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item()) # 정확도 88.9% 학습이 잘되었다고 볼수있다.

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = linear(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.8894999623298645
Label:  8
Prediction:  3


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
